# Lesson 6: Database TODO list: Updating tables

### Goal: make a persistent TODO list.

### Overview

This example program is superficially similar to lesson 4's TODO list. However, instead of storing the TODO list in a list value on the client, this program stores it in the database. So, there are two important differences:

1. Todo list items are persistent in the database and will survive if the Links program terminates and is restarted. (By the same token, they could also be accessed through the database interactive interface or by other applications with access to the database.)
2. Todo list items are centralized and different copies of the todo list program running in different browser windows can see the same data.

The user interface for this version of the TODO list program is the same as before, so we won't explain it again. The main differences are in how the list is displayed and changed.

In `showList`, instead of using a list comprehension over an in-memory list, we use a query: `query {for (item <-- items) [item]}`. This generates a simple SQL query that just returns all of the list items. Because `showList` queries the database, we annotate it `server` so that this happens on the server.

The actions for the insert and remove buttons are also different: they simply call the `add` and `remove` functions. These functions use Links's syntax for database table:

`insert TABLE_NAME values [(COLUMN1_NAME=VALUE1, COLUMN2_NAME=VALUE2, ...)];`

And

`delete (r <-- TABLE_NAME) where (r.COLUMN1_NAME=VALUE1, r.COLUMN2_NAME=VALUE2, ...);`

As before, `add` and `remove` are annotated server so that these will be performed on the server.

Links also supports `update` syntax, for example:

`update (i <-- items)
   where (i.name == oldname)
    set (name=newname)`

renames an item from `oldname` to `newname`.

### Task

To make this web page work, add an argument in the `remove` call in `showList`.

In [ ]:
var db = database "links";
var items = table "todo" with (name : String) from db;
  
fun showList() server {
   page
    <html>
     <body>
      <form l:action="{add(item)}" method="POST">
        <input l:name="item"/>
        <button type="submit">Add item</button>
      </form>
      <table>
       {for (item <- query {for (item <-- items) [item]})
          <tr><td>{stringToXml(item.name)}</td>
              <td><form l:action="{remove( # Fill argument )}" method="POST">
                   <button type="submit">Done</button>
                  </form>
              </td>
          </tr>}
       </table>
      </body>
    </html>
}

In [ ]:
fun add(name) server {
   insert items values [(name=name)];
   showList()
}
  
fun remove(name) server {
   delete (r <-- items) where (r.name == name);
   showList()
}

In [ ]:
fun mainPage () {
    showList()
}

mainPage()

### Additional Exercises

1. What happens if the `server` annotation is removed from `showList`, `add` or `remove`?


2. What happens if you replace the `server` annotation with `client` in the above functions?


3. Using `update`, modify the code to allow renaming an existing item.


4. This version of the TODO list uses `l:action` to handle the form responses by POSTing to the server. Some server communication is unavoidable because we need to get data from the database, or update the database, but it should be possible to rewrite this program to use `l:onsubmit` to avoid completely rebuilding the page whenever the a button is clicked, using a similar approach to the client-side form in lessons 3 or 5. Modify the code to work this way.
